In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

from face_detections import face_detect, gray_scale, cv_face_detect

In [ ]:
os.path.dirname(os.path.abspath('__file__'))

In [ ]:
class CFG:
    data_dir = os.getcwd() + '/../input/data/train' # /opt/ml/project/DataProcessing/../input/data/train
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'

In [ ]:
num2class = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']
class2num = {k:v for v, k in enumerate(num2class)}
print(class2num)

df = pd.read_csv(CFG.df_path)
df.head()

In [ ]:
def image_path(idx, normal=6):
    return CFG.img_dir +'/' + df['path'][idx] + '/' + num2class[normal] + '.jpg'

print(image_path(0, 0))
image = cv2.imread(image_path(0, 5))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
image = cv2.imread(image_path(1245, 5))

face_image = face_detect(image, gray_scale)
if face_image is None:
    face_image = cv_face_detect(image)
# face_image = cv_face_detect(image)
    
plt.imshow(face_image)
plt.show()
print(face_image.shape)

In [ ]:
image = cv2.imread(image_path(1156, 5))
h, w, c = image.shape
print(w, h)
image = image[64:384][55:256]
plt.imshow(image)
plt.show()